# Library

In [1]:
import pandas as pd
import numpy as np
import os
import re
import collections
#from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
import itertools 

# Functions

In [2]:
def read_texts(path):
    data = []
    file_name = os.listdir(path)

    for name in file_name:
        if name.endswith('.txt'):
            with open(path + name) as f:
                text = f.read()
                data.append({'nombre':name.replace('.txt',''), 'texto':text})

    df = pd.DataFrame(data)
    return df

In [3]:
# def filter_byindex(x,y):
#     #selectors = [x for x in col2]
#     return list(itertools.compress(x,y))

In [74]:
def check_len(x,y):
    if x==y:
        return 0
    else:
        return 1

# Load Data

### Testing

220 clinical cases.

In [4]:
testing_abbr = pd.read_csv("../datasets/testing_set/clinical_cases.abbreviations.testing_set.tsv", sep = '\t')
testing_met = pd.read_csv("../datasets/testing_set/clinical_cases.metadata.testing_set.tsv", sep = '\t')
testing_rel = pd.read_csv("../datasets/testing_set/clinical_cases.relations.testing_set.tsv", sep = '\t')

In [5]:
testing_met.head()

,# Document_ID,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1130-01082006000700014-1.txt,1.txt,1130-0108,2006-07-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-01082007000300006-7.txt,7.txt,1130-0108,2007-03-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S1134-80462009000100005-1.txt,1.txt,1134-8046,2009-02-01,Revista de la Sociedad Española del Dolor v.1...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S1137-66272014000300015-1.txt,1.txt,1137-6627,2014-12-01,Anales del Sistema Sanitario de Navarra v.37 ...,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0365-66912004001200011-1.txt,1.txt,0365-6691,2004-12-01,Archivos de la Sociedad Española de Oftalmolog...,http://scielo.isciii.es/scielo.php?script=sci_...


In [6]:
testing_rel = testing_rel.reset_index()

In [7]:
testing_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [8]:
testing_rel = testing_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [9]:
testing_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S0211-69952013000500019-1,SHORT_FORM,3739,ARA II,SHORT-LONG,LONG_FORM,3695,3737,antagonista del receptor de angiotesina II
1,S0211-69952013000500019-1,SHORT_FORM,2793,PCR,SHORT-LONG,LONG_FORM,2798,2837,reacción en cadena de enzima polimerasa
2,S0365-66912004000600008-1,SHORT_FORM,406,AV,SHORT-LONG,LONG_FORM,390,404,agudeza visual
3,S0211-69952012000500025-1,SHORT_FORM,945,angio-TAC,SHORT-LONG,LONG_FORM,908,943,angiotomografía computarizada axial
4,S1130-05582017000200122-1,SHORT_FORM,940,RMN,SHORT-LONG,LONG_FORM,910,938,resonancia magnética nuclear


In [10]:
testing_rel.Relation_type.unique()

array(['SHORT-LONG', 'SHORT-NESTED', 'NESTED-LONG'], dtype=object)

In [11]:
testing_abbr = testing_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [12]:
testing_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0004-06142010000500014-1,2037,2044,16SrRNA,16s ribosomal rna,16s ribosomal rno
1,S0004-06142010000500014-1,1349,1351,M.,mycobacterium,mycobacterium
2,S0004-06142010000500014-1,1339,1342,PCR,polymerase chain reaction,polymerase chain reaction
3,S0004-06142010000500014-1,611,615,BHCG,beta-human chorionic gonadotropin,beta-humar chorionic gonadotropin
4,S0004-06142010000500014-1,594,597,CEA,carcinoembrionary antigen,carcinoembrionary antiger


In [13]:
testing_raw = read_texts("../datasets/testing_set/testing_set.raw_text/")

In [14]:
testing_raw.head()

,nombre,texto
0,S1139-76322009000700016-1,Paciente de sexo femenino de 13 años y 7 meses...
1,S0210-48062007000700015-1,Varón de 72 años con antecedentes personales d...
2,S0212-71992005000500009-1,"Varón de 81 años, con antecedentes de fibrilac..."
3,S0365-66912004001200011-1,Paciente varón de 52 años que acudió a urgenci...
4,S1130-01082009000900015-1,Varón de 54 años con episodios de pancreatitis...


### Backgound

2879 clinical cases. 220 will be use for clinical evaluation

In [15]:
back_met = pd.read_csv("../datasets/background_test/clinical_cases.metadata.background_set.tsv", sep = '\t')

In [16]:
back_met = back_met.rename(columns = {'# Document_ID': 'doc_id'})

In [17]:
back_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S0325-00752010000100014.txt,1.txt,0325-0075,02/2010,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...
1,S0325-00752013000200014.txt,1.txt,0325-0075,04/2013,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...
2,S0325-00752011000400017.txt,1.txt,0325-0075,08/2011,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...
3,S0325-00752013000600022.txt,1.txt,0325-0075,12/2013,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...
4,S0325-00752008000500013.txt,1.txt,0325-0075,10/2008,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...


In [18]:
back_raw = read_texts("../datasets/background_test/background_test_set/")

In [19]:
back_raw.head()

,nombre,texto
0,S0212-71992004000800006-1,Mujer de 57 años de edad con antecedentes pers...
1,S0716-10182014000100007-1,Se recibió en el Hospital Veterinario Puente A...
2,S1130-01082009000200016-1,"Mujer de 47 años de edad, sin antecedentes de ..."
3,S0716-10182015000400016-1,"Escolar de 11 años de edad, género femenino, s..."
4,S1130-01082004001200010-2,"Varón de 25 años, remitido a nuestro Servicio ..."


### Development

146 clinical cases

In [20]:
dev_abbr = pd.read_csv("../datasets/development_set/clinical_cases.abbreviations.development_set.tsv", sep = '\t')
dev_met = pd.read_csv("../datasets/development_set/clinical_cases.metadata.development_set.tsv", sep = '\t')
dev_rel = pd.read_csv("../datasets/development_set/clinical_cases.relations.development_set.tsv", sep = '\t')

In [21]:
dev_met = dev_met.rename(columns = {'# Document_ID': 'doc_id'})

In [22]:
dev_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1130-14732005000300004-1.txt,1.txt,1130-1473,2005-06-01,Neurocirugía v.16 n.3 2005,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-01082008000200009-1.txt,1.txt,1130-0108,2008-02-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S1137-66272012000300021-1.txt,1.txt,1137-6627,2012-12-01,Anales del Sistema Sanitario de Navarra v.35 ...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S1699-695X2016000200010-1.txt,1.txt,1699-695X,2016-06-01,Revista Clínica de Medicina de Familia v.9 n....,http://scielo.isciii.es/scielo.php?script=sci_...
4,S1130-01082007001100009-1.txt,1.txt,1130-0108,2007-11-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...


In [23]:
dev_rel = dev_rel.reset_index()

In [24]:
dev_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [25]:
dev_rel = dev_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [26]:
dev_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1888-75462015000400006-1,SHORT_FORM,1436,AP,SHORT-LONG,LONG_FORM,1419,1434,anteroposterior
1,S0210-56912009000700006-2,SHORT_FORM,876,angio-RMN,SHORT-LONG,LONG_FORM,858,874,angiorresonancia
2,S0210-56912009000700006-2,SHORT_FORM,819,RMN,SHORT-LONG,LONG_FORM,789,817,resonancia magnética nuclear
3,S1698-44472005000300015-1,SHORT_FORM,209,ATM,SHORT-LONG,LONG_FORM,177,207,articulación témporomandibular
4,S1698-44472005000300015-1,SHORT_FORM,1125,TC,SHORT-LONG,LONG_FORM,1099,1123,tomografía computerizada


In [27]:
dev_abbr = dev_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [28]:
dev_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S1130-14732005000300004-1,1216,1218,C3,tercera vértebra cervical,tercero vértebra cervical
1,S1130-14732005000300004-1,717,719,C2,segunda vértebra cervical,segundo vértebra cervical
2,S1130-14732005000300004-1,3191,3193,C3,tercera vértebra cervical,tercero vértebra cervical
3,S1130-14732005000300004-1,2867,2869,C3,tercera vértebra cervical,tercero vértebra cervical
4,S1130-14732005000300004-1,2862,2864,C2,segunda vértebra cervical,segundo vértebra cervical


In [29]:
dev_raw = read_texts("../datasets/development_set/development_set.raw_text/")

In [30]:
dev_raw.head()

,nombre,texto
0,S1139-76322017000200010-1,Niña de dos años y diez meses con antecedentes...
1,S0365-66912005001100008-1,Se presenta el caso de un varón de 45 años que...
2,S1130-01082006001000017-1,Mujer de 42 años de edad con antecedentes pers...
3,S0212-71992005001200008-1,Paciente de 57 años con ingresos hospitalarios...
4,S0365-66912007000300010-1,Paciente de 33 años que el 20-08-05 es traslad...


### Trainning

318 clinical cases

In [31]:
train_abbr = pd.read_csv("../datasets/trainning_set/clinical_cases.abbreviations.training_set.tsv", sep = '\t')
train_met = pd.read_csv("../datasets/trainning_set/clinical_cases.metadata.training_set.tsv", sep = '\t')
train_rel = pd.read_csv("../datasets/trainning_set/clinical_cases.relations.training_set.tsv", sep = '\t')

In [32]:
train_met = train_met.rename(columns = {'# Document_ID': 'doc_id'})

In [33]:
train_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1139-76322015000500009-1.txt,1.txt,1139-7632,2015-12-01,Pediatría Atención Primaria v.17 n.68 2015,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-05582008000400007-2.txt,2.txt,1130-0558,2008-08-01,Revista Española de Cirugía Oral y Maxilofacia...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S0210-48062006000100012-1.txt,1.txt,0210-4806,2006-01-01,Actas Urológicas Españolas v.30 n.1 2006,http://scielo.isciii.es/scielo.php?script=sci_...
3,S0213-12852003000500002-1.txt,1.txt,0213-1285,2003-10-01,Avances en Odontoestomatología v.19 n.5 2003,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0212-71992005000400007-1.txt,1.txt,0212-7199,2005-04-01,Anales de Medicina Interna v.22 n.4 2005,http://scielo.isciii.es/scielo.php?script=sci_...


In [34]:
train_rel = train_rel.reset_index()

In [35]:
train_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [36]:
train_rel = train_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [37]:
train_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1130-01082009000400014-1,SHORT_FORM,476,NPT,SHORT-LONG,LONG_FORM,454.0,474.0,nutrición parenteral
1,S1130-63432016000100009-1,SHORT_FORM,614,NIHSS,SHORT-LONG,LONG_FORM,621.0,662.0,National Institute of Health Stroke Scale
2,S1139-76322017000200007-1,SHORT_FORM,1145,CMV,SHORT-LONG,LONG_FORM,1128.0,1143.0,citomegalovirus
3,S1139-76322017000200007-1,SHORT_FORM,1243,VSG,SHORT-LONG,LONG_FORM,1206.0,1241.0,velocidad de sedimentación globular
4,S1139-76322017000200007-1,SHORT_FORM,1300,IGRA,SHORT-LONG,LONG_FORM,1267.0,1298.0,interferon-gamma release assays


In [38]:
train_abbr = train_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [39]:
train_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0210-48062004000500008-1,1650,1652,ml,mililitro,mililitro
1,S0210-48062004000500008-1,708,709,l,litro,litro
2,S0210-48062004000500008-1,704,707,mEq,miliequivalente,miliequivalente
3,S0210-48062004000500008-1,677,681,pCO2,presión parcial de co2,presión parcial de co2
4,S0210-48062004000500008-1,2287,2290,HLA,human leucocyte antigen,human leucocyte antiger


In [180]:
train_raw = read_texts("../datasets/trainning_set/training_set.raw_text/")

In [181]:
train_raw = train_raw.rename(columns = {'nombre': 'doc_id'})

In [182]:
train_raw.head()

,doc_id,texto
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que..."
2,S0004-06142008000700015-2,Varón de 33 años fumador de un paquete de ciga...
3,S0210-56912006000800008-1,"Hombre de 42 años, bebedor de más de 100 g de ..."
4,S0376-78922009000300010-1,Paciente de 18 años de edad que 5 meses antes ...


### Sample test

15 clinical cases

In [43]:
sample_abbr = pd.read_csv("../datasets/sample_set/clinical_cases.abbreviations.sample_set.tsv", sep = '\t')
sample_met = pd.read_csv("../datasets/sample_set/clinical_cases.metadata.sample_set.tsv", sep = '\t')
sample_rel = pd.read_csv("../datasets/sample_set/clinical_cases.relations.sample_set.tsv", sep = '\t')

In [44]:
sample_met = sample_met.rename(columns = {'# Document_ID': 'doc_id'})

In [45]:
sample_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S0004-06142006000900015-1,1,0004-0614,2006-11-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
1,S0004-06142006000600015-1,1,0004-0614,2006-08-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S0004-06142007000700014-1,1,0004-0614,2007-09-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S0004-06142007000900013-1,1,0004-0614,2007-11-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0004-06142006000200014-1,1,0004-0614,2006-03-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...


In [46]:
sample_rel = sample_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [47]:
sample_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A_EndOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S0004-06142006000700014-1,SHORT_FORM,926,929,CEA,SHORT-LONG,LONG_FORM,896,924,Antígeno Carcino Embrionario
1,S0004-06142005001000011-1,SHORT_FORM,1626,1629,ROT,SHORT-LONG,LONG_FORM,1600,1624,reflejos osteotendinosos
2,S0004-06142005001000011-1,SHORT_FORM,1715,1718,RMN,SHORT-LONG,LONG_FORM,1685,1713,resonancia magnética nuclear
3,S0004-06142005001000011-1,SHORT_FORM,1663,1666,RCP,SHORT-LONG,LONG_FORM,1639,1661,reflejo cutaneoplantar
4,S0004-06142005001000011-1,SHORT_FORM,1808,1811,LCR,SHORT-LONG,LONG_FORM,1783,1806,líquido cefalorraquídeo


In [48]:
sample_abbr = sample_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [49]:
sample_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0004-06142005001000011-1,1034,1036,Kg,kilogramo,kilogramo
1,S0004-06142005001000011-1,1031,1033,mg,miligramo,miligramo
2,S0004-06142005001000011-1,196,199,IgA,inmunoglobulina a,inmunoglobulina a
3,S0004-06142005001000011-1,2057,2060,LCR,líquido cefalorraquídeo,líquido cefalorraquídeo
4,S0004-06142005001000011-1,1594,1598,EEII,extremidades inferiores,extremidad inferior


In [50]:
sample_raw = read_texts("../datasets/sample_set/sample_set.raw_text/")

In [51]:
sample_raw.head()

,nombre,texto
0,S0004-06142006000600014-1,"Paciente varón, de 40 años de edad, con antece..."
1,S0004-06142006000300015-1,Paciente de 50 años con antecedente de litiasi...
2,S0004-06142007000900013-1,Presentamos el caso de un recién nacido de tre...
3,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...
4,S0004-06142006000700013-1,"Paciente varón, de 63 años de edad, mestizo, d..."


# Sub-track 1

## Found abbreviations

### Regex

In [52]:
#patron = r'\(([A-Z]{2,8})\)'
#patron1 = r'\s[A-Z]{1,3}\s'
#patron2 = r'\s[a-z]{1,3}\s'
patron3 = r'[A-Z]{2,8}'
patron4 = r'\s[a-z]{1,2}\s'
patron5 = r'[a-z]+\-[A-Z]{1,8}'
patron6 = r'[a-z]+\/[a-z]+'
patron7 = r'[A-Z]?[a-z]{1,4}[A-Z]+[a-z]*[1-9]*'
patron8 = r'\/[a-z]*[A-Z]*'


# create a list with them
regexes = [ patron3, patron4, patron5, patron6, patron7,patron8]
for i in regexes:
    generic_re = re.compile("%s|%s|%s|%s|%s|%s" % (patron3, patron4, patron5, patron6, patron7,patron8))

#### Solution 1

In [53]:
# todos = []
# for _,row in train_raw.iterrows():
#     texto = row['texto']
#     id_texto = row['nombre']
#     ab_found= set(generic_re.findall(texto))
#     #print(texto,id_texto, ab_found)
#     for item in ab_found:
#         abr = {}
#         abr = {'id_texto':id_texto,'abre':item }
#         #print(abr)
#         #abr[id_texto] = {item} | abr.get(id_texto) if abr.get(id_texto) else {item}
#         todos.append(abr)

#todos

In [54]:
# df_abr = pd.DataFrame(todos)

In [55]:
# df_abr.head()


In [56]:
#df_abr = pd.DataFrame(abr.items(), columns=['# Document_ID', 'abbreviation_m'])

#### Solution 2 (best one)

In [57]:
#nltk.download('stopwords')
swords = list(set(stopwords.words('spanish')))

Get Short Formns with a regex in each text

In [183]:
train_raw['abrev'] = train_raw['texto'].map(lambda x: generic_re.findall(x))

In [59]:
# train_raw['offse'] = train_raw['texto'].map(lambda x: generic_re.search(x).span())

Get offsets of the Short Forms founded in the text

In [184]:
train_raw['offse'] = train_raw['texto'].map(lambda x: [(m.start(0), m.end(0)) for m in re.finditer(generic_re, x)])

In [185]:
train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [i.strip() for i in x])

Filter Short Forms to delete ones which are stopwords, get their index too for filter offsets lists later

In [186]:
train_raw['abrev_index'] = train_raw['abrev'].apply(lambda x: [x.index(i) for i in x if i not in swords])

In [187]:
train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [i for i in x if i not in swords])

Filter offsets lists by index

In [188]:
train_raw['offse'] = train_raw.apply(lambda x: [x['offse'][i] for i in x['abrev_index']], axis = 1)

In [189]:
train_raw.head()

,doc_id,texto,abrev,offse,abrev_index
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,"[RM, PAAF]","[(789, 791), (1006, 1010)]","[24, 31]"
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...","[/mm, /mm, mg/dl, mg/dl, mg/dl, mg/dl, LDH, UI...","[(1056, 1059), (1056, 1059), (1247, 1252), (12...","[36, 36, 44, 44, 44, 44, 50, 51, 52, 53, 51, 5..."
2,S0004-06142008000700015-2,Varón de 33 años fumador de un paquete de ciga...,[],[],[]
3,S0210-56912006000800008-1,"Hombre de 42 años, bebedor de más de 100 g de ...","[g, mg/dl, mg/dl, mg/dl, AST, /l, ALT, /l, GGT...","[(40, 43), (654, 659), (654, 659), (654, 659),...","[3, 23, 23, 23, 26, 27, 28, 27, 30, 31, 32, 39..."
4,S0376-78922009000300010-1,Paciente de 18 años de edad que 5 meses antes ...,[],[],[]


Check abrev and offse columns has the same length

In [190]:
train_raw['abrev_len'] =train_raw['abrev'].str.len()
train_raw['offse_len'] =train_raw['offse'].str.len()

In [191]:
train_raw.head()

,doc_id,texto,abrev,offse,abrev_index,abrev_len,offse_len
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,"[RM, PAAF]","[(789, 791), (1006, 1010)]","[24, 31]",2,2
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...","[/mm, /mm, mg/dl, mg/dl, mg/dl, mg/dl, LDH, UI...","[(1056, 1059), (1056, 1059), (1247, 1252), (12...","[36, 36, 44, 44, 44, 44, 50, 51, 52, 53, 51, 5...",43,43
2,S0004-06142008000700015-2,Varón de 33 años fumador de un paquete de ciga...,[],[],[],0,0
3,S0210-56912006000800008-1,"Hombre de 42 años, bebedor de más de 100 g de ...","[g, mg/dl, mg/dl, mg/dl, AST, /l, ALT, /l, GGT...","[(40, 43), (654, 659), (654, 659), (654, 659),...","[3, 23, 23, 23, 26, 27, 28, 27, 30, 31, 32, 39...",25,25
4,S0376-78922009000300010-1,Paciente de 18 años de edad que 5 meses antes ...,[],[],[],0,0


Check both lists have the same lenght

In [192]:
train_raw['len_check'] = train_raw.apply(lambda row: check_len(row['abrev_len'],row['offse_len']),axis = 1)

Delete 3 rows with different lenghts (check it later)

In [193]:
train_raw.shape

(318, 8)

In [194]:
train_raw[train_raw['len_check'] != 1].shape

(318, 8)

In [195]:
train_raw = train_raw[train_raw['len_check'] != 1]

In [196]:
train_raw.shape

(318, 8)

Separate elements lists in different rows

In [197]:
mine = train_raw.explode('abrev')
mine = mine.explode('offse')

In [198]:
mine = mine[['doc_id', 'texto', 'abrev', 'offse']]

Separate offsets tuples in different columns

In [199]:
mine[['startOffset', 'endOffset']] = pd.DataFrame(mine['offse'].tolist(), index=mine.index) 

Delete null values, and change ttype to integer for offsets columns

In [200]:
mine.shape

(86869, 6)

In [201]:
mine.dropna(subset=['startOffset', 'endOffset'], inplace = True)

In [202]:
mine.shape

(86851, 6)

In [212]:
mine['startOffset'] = mine['startOffset'].astype(int)
mine['endOffset'] = mine['endOffset'].astype(int)

In [213]:
mine = mine[['doc_id', 'texto', 'abrev', 'startOffset', 'endOffset']]

In [214]:
mine.head()

,doc_id,texto,abrev,startOffset,endOffset
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,RM,789,791
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,RM,1006,1010
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,PAAF,789,791
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,PAAF,1006,1010
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que...",/mm,1056,1059


Check dataframes for one text

In [206]:
mine[mine['doc_id'] == 'S1130-05582012000300005-1']

,doc_id,texto,abrev,startOffset,endOffset
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,RM,789,791
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,RM,1006,1010
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,PAAF,789,791
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,PAAF,1006,1010


In [207]:
train_abbr[train_abbr['doc_id'] == 'S1130-05582012000300005-1']

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
2112,S1130-05582012000300005-1,1006,1010,PAAF,punción aspiración con aguja fina,punción aspiración con aguja fino
2113,S1130-05582012000300005-1,789,791,RM,resonancia magnética,resonancia magnético
2114,S1130-05582012000300005-1,1345,1347,cm,centímetro,centímetro
2115,S1130-05582012000300005-1,1304,1305,g,gramo,gramo
2116,S1130-05582012000300005-1,1217,1219,cm,centímetro,centímetro


#### Join with train_abr to see how many abbreviations I have found

In [215]:
matches = train_abbr[['doc_id', 'Abbreviation', 'StartOffset', 'EndOffset']].merge(mine, how = 'left', left_on=['doc_id', 'Abbreviation','StartOffset', 'EndOffset'], right_on = ['doc_id', 'abrev','startOffset', 'endOffset'])

In [216]:
matches.head()

,doc_id,Abbreviation,StartOffset,EndOffset,texto,abrev,startOffset,endOffset
0,S0210-48062004000500008-1,ml,1650,1652,NaN,NaN,NaN,NaN
1,S0210-48062004000500008-1,l,708,709,NaN,NaN,NaN,NaN
2,S0210-48062004000500008-1,mEq,704,707,Paciente de 29 años de edad que acude al Servi...,mEq,704.0,707.0
3,S0210-48062004000500008-1,pCO2,677,681,Paciente de 29 años de edad que acude al Servi...,pCO2,677.0,681.0
4,S0210-48062004000500008-1,HLA,2287,2290,Paciente de 29 años de edad que acude al Servi...,HLA,2287.0,2290.0


In [218]:
train_abbr.shape

(4260, 6)

In [219]:
mine.shape

(86851, 5)

In [225]:
matches.shape

(7303, 8)

In [228]:
matches[matches[['abrev','startOffset', 'endOffset']].isnull().all(1)].shape

(3153, 8)

Check how many abbreviations are detected

In [229]:
#matches = matches[['texto', 'doc_id','Abbreviation', 'abrev']]

In [230]:
matches = matches.rename(columns = {'Abbreviation': 'abbr_oficial', 'abrev': 'my_abbr'})

In [231]:
matches['my_abbr'].fillna('0', inplace = True)

In [232]:
matches['compare'] = np.where(matches['abbr_oficial'] == matches['my_abbr'], 1, 0)

In [ ]:
# matches['percentage'] = matches.groupby(['doc_id']).apply(lambda x: x['compare'].sum())#/matches.groupby(['doc_id'])['doc_id'].count()*100

In [233]:
serie = matches.groupby(['doc_id']).apply(lambda x: x['compare'].sum())/matches.groupby(['doc_id'])['doc_id'].count()*100

In [234]:
aciertos = serie.to_frame(name = 'hits')

In [235]:
aciertos['hits'] = aciertos['hits'].apply(lambda x: int(x)) 

In [236]:
aciertos.hits.value_counts(normalize = True).mul(100)

33     7.382550
100    6.040268
0      5.369128
50     3.691275
80     3.691275
         ...   
68     0.335570
73     0.335570
78     0.335570
79     0.335570
48     0.335570
Name: hits, Length: 77, dtype: float64

In [238]:
#sorted(aciertos.hits.unique().tolist())